In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import duckdb
from uuid import uuid4
conn = duckdb.connect()

In [13]:
query = """
CREATE TABLE IF NOT EXISTS conversation (
    convo_id VARCHAR(36) PRIMARY KEY,
    content TEXT NOT NULL,
    role VARCHAR(10) CHECK (role IN ('user', 'assistant')),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    session_id VARCHAR(36)
);
"""
conn.execute(query).fetchall()

[]

In [14]:
conn.execute("SHOW TABLES;").df()

,name
0,conversation


In [15]:
query = """
INSERT INTO conversation (convo_id, content, role) VALUES (?, ?, ?);
"""

conn.execute(query, [str(uuid4()), "Hello, world!", "user"]).fetchall()

[(1,)]

In [16]:
query = """
SELECT * FROM conversation;
"""
conn.execute(query).df()

,convo_id,content,role,created_at,updated_at,session_id
0,d598b3ed-564e-4050-877f-67d15299f8f9,"Hello, world!",user,2025-11-11 22:05:54.905936,2025-11-11 22:05:54.905936,None


In [17]:
convo_id = conn.execute("SELECT convo_id FROM conversation LIMIT 1;").fetchone()[0]
query = """
UPDATE conversation
SET content = ?,
updated_at = CURRENT_TIMESTAMP
WHERE convo_id = ?;
"""

conn.execute(query, ["Holla world!", convo_id]).fetchall()

[(1,)]

In [18]:
query = """
SELECT * FROM conversation;
"""
conn.execute(query).df()

,convo_id,content,role,created_at,updated_at,session_id
0,d598b3ed-564e-4050-877f-67d15299f8f9,Holla world!,user,2025-11-11 22:05:54.905936,2025-11-11 22:05:54.938240,None


In [19]:
query = """
DELETE FROM conversation WHERE convo_id = ?;
"""

conn.execute(query, [convo_id]).fetchall()

[(1,)]

In [20]:
query = """
SELECT * FROM conversation;
"""
conn.execute(query).df()

,convo_id,content,role,created_at,updated_at,session_id


In [23]:
from uuid import uuid4
from datetime import datetime
from pydantic import BaseModel, Field
from typing import List, Optional, Any, Optional, Dict, Literal
from enum import StrEnum, IntEnum, Enum

class ConversationRole(StrEnum):
    USER = "user"
    ASSISTANT = "assistant"

class Artifact(BaseModel):
    type:Literal["sql", "data", "html"]
    data:Any

class Conversation(BaseModel):
    conversation_id:str = Field(default_factory=lambda: str(uuid4()))
    content:str
    role:ConversationRole
    created_at:datetime = Field(default_factory=datetime.now)
    updated_at:datetime = Field(default_factory=datetime.now)
    artifacts:Optional[List[Artifact]] = Field(default=None)
    session_id:Optional[str] = Field(default=None)

Conversation(content="Hi", role=ConversationRole.USER)

Conversation(conversation_id='c2488c19-1eb3-442c-a1da-778b1d91837a', content='Hi', role=<ConversationRole.USER: 'user'>, created_at=datetime.datetime(2025, 11, 11, 22, 34, 35, 99163), updated_at=datetime.datetime(2025, 11, 11, 22, 34, 35, 99163), artifacts=None, session_id=None)